# Topic Modeling
### Without Preprocessing

In [1]:
import json
import os
import random
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
import spacy
# import coreferee
# nlp = spacy.load('en_core_web_trf')
# nlp.add_pipe('coreferee')
nlp = spacy.load("en_core_web_sm")

2022-08-28 17:59:26.215100: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-28 17:59:26.215146: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


#### Define Parameters

In [30]:
n_components = 50 #Topics To generate
n = 40 #Transcripts to use
max_df=20
min_df=3
n_terms = 10

## Load Data

In [3]:
# pos = ["CCONJ","NUM","PRON","PROPN","PUNCT","SCONJ","SYM","X"]
# pos = ["ADJ","ADP","ADV","AUX","CCONJ","DET","INTJ","NOUN","NUM","PART","PRON","PROPN","PUNCT","SCONJ","SYM","VERB","X"]
pos = ["ADJ","AUX","CCONJ","DET","INTJ","NUM","PART","PRON","PROPN","PUNCT","SCONJ","SYM","X"]
def clean_text(text, coref = False, lemma = True, pos_exclusions = pos):
    doc = nlp(text)

    if coref:
        new_text = ""
        for i  in range(len(doc)):
            tokens = doc[i]
            if doc._.coref_chains.resolve(tokens):
                new_text += doc._.coref_chains.resolve(tokens)[0].text
            else:
                new_text += tokens.text
            new_text+= " "
        doc = nlp(new_text)
    text = ""
    for token in doc:
        if token.pos_ not in pos_exclusions:
            text += token.lemma_ if lemma else token.text
            text += " "
    return text

In [4]:
def create_df_multi(n=25):
    list_of_text = []
    dir = 'data/aligned data/c=4'

    files = [filename for filename in os.listdir(dir)]
    sampled_files = random.choices(files,k=n)

    for filename in sampled_files:
        # choice = random.choice(os.listdir(dir))
        # print(choice)
        f = open(dir+'/'+filename)
        data = json.load(f)
        choice = random.choice(os.listdir(dir))
        # print(choice)
        f = open(dir+'/'+choice)
        data = json.load(f)

        for x in data:
            for y in x['TURNS']:
                text = ' '.join(y['UTTERANCES'])

                text = clean_text(text)
                list_of_text.append(text)
    df = pd.DataFrame(list_of_text)
    return df

In [5]:
df = create_df_multi(n)
print(df.shape)
df.head()

(105048, 1)


,0
0,welcome evening to show bunch of ass voice act...
1,welcome back so first foremost get couple anno...
2,so
3,slightly
4,go worry about want apologize in advance to fo...


## Vectorize Data



- Count Vectorizer
- Tfidf Vectorizer

Use both with each model

In [6]:
# tfidf=TfidfVectorizer(stop_words='english',max_df=.7,min_df=2,token_pattern=r'(?u)\b[A-Za-z]+\b')

tfidf = TfidfVectorizer(stop_words='english', 
    max_df=max_df,
    min_df=min_df,
    token_pattern=r'(?u)\b[A-Za-z]+\b'
    )
tfidf_sparse = tfidf.fit_transform(df[0])
print(tfidf_sparse.shape)
tfidf_df = pd.DataFrame(tfidf_sparse.toarray().transpose(),
                   index=tfidf.get_feature_names_out())
tfidf_df.tail()

(105048, 4191)


,0,1,2,3,4,5,6,7,8,9,...,105038,105039,105040,105041,105042,105043,105044,105045,105046,105047
zigzag,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zip,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zombie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zone,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zoo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
cv = CountVectorizer(stop_words='english', 
    max_df=max_df,
    min_df=min_df,
    token_pattern=r'(?u)\b[A-Za-z]+\b'
    )
cv_sparse = cv.fit_transform(df[0])
print(cv_sparse.shape)
cv_df = pd.DataFrame(cv_sparse.toarray().transpose(),
                   index=cv.get_feature_names_out())
cv_df.tail()

(105048, 4191)


,0,1,2,3,4,5,6,7,8,9,...,105038,105039,105040,105041,105042,105043,105044,105045,105046,105047
zigzag,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zip,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zombie,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zone,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zoo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Define Output Functions

In [21]:
def print_top_terms(n_components, topics, terms, n_terms = 10):
    for x in range(n_components):
        topic = x
        components = topics[:,topic]
        top_term_indices = components.argsort()[-n_terms:]
        top_terms = np.array(terms)[top_term_indices]
        
        print(f'Topic {x}:\t{top_terms.tolist()}')


## PCA

In [31]:
pca = PCA(n_components=n_components)
topics = pca.fit_transform(cv_df)
print_top_terms(n_components=n_components, topics=topics, terms=cv.get_feature_names_out())

Topic 0:	['practitioner', 'greed', 'hoard', 'narrowly', 'mercy', 'contemplate', 'cling', 'creator', 'ceremony', 'looter']
Topic 1:	['combine', 'nostril', 'necessity', 'ware', 'sage', 'breeze', 'drape', 'ceremony', 'port', 'looter']
Topic 2:	['conclude', 'installation', 'inhale', 'invoke', 'parameter', 'eternity', 'completion', 'obligation', 'scribe', 'recipient']
Topic 3:	['flying', 'hike', 'manipulation', 'tradition', 'knack', 'meditate', 'childhood', 'truly', 'impression', 'headmaster']
Topic 4:	['potential', 'devise', 'abdoman', 'orcs', 'puzzle', 'reign', 'demand', 'devour', 'impression', 'headmaster']
Topic 5:	['kneel', 'brighten', 'intensity', 'atop', 'chip', 'stain', 'account', 'stillness', 'entryway', 'sanctuary']
Topic 6:	['recede', 'intersection', 'amphitheater', 'desperation', 'crossroad', 'speech', 'prayer', 'bond', 'erathis', 'bench']
Topic 7:	['commotion', 'snuck', 'cobra', 'midst', 'boon', 'purely', 'follower', 'masquerade', 'ti', 'yuan']
Topic 8:	['unfold', 'user', 'orig

In [32]:
pca = PCA(n_components=n_components)
topics = pca.fit_transform(tfidf_df)
print_top_terms(n_components=n_components, topics=topics, terms=tfidf.get_feature_names_out())

Topic 0:	['hex', 'impression', 'marching', 'sailor', 'okay', 'abomination', 'sneeze', 'advisor', 'socket', 'eyeball']
Topic 1:	['user', 'vomit', 'advice', 'investigate', 'tearfully', 'sailor', 'pen', 'invisibility', 'locket', 'grieve']
Topic 2:	['advisor', 'bless', 'stat', 'proof', 'tally', 'score', 'text', 'lycan', 'theater', 'locket']
Topic 3:	['represent', 'vomit', 'tearfully', 'lycan', 'warn', 'gone', 'invisibility', 'grieve', 'pen', 'theater']
Topic 4:	['bore', 'slice', 'sizzle', 'cousin', 'pen', 'locket', 'grieve', 'disarm', 'theater', 'gingerly']
Topic 5:	['warn', 'lately', 'sure', 'mimic', 'disarm', 'locket', 'invisibility', 'pen', 'gingerly', 'lycan']
Topic 6:	['warn', 'sizzle', 'tearfully', 'locket', 'okay', 'warhammer', 'yesterday', 'lately', 'pen', 'invisibility']
Topic 7:	['accord', 'mimic', 'bakery', 'toy', 'locket', 'queen', 'y', 'text', 'tary', 'pen']
Topic 8:	['opposite', 'misfire', 'grieve', 'horribly', 'sizzle', 'lycan', 'invisibility', 'yesterday', 'gingerly', 'tary

## LDA

In [33]:
lda = LatentDirichletAllocation(n_components=n_components)
topics = lda.fit_transform(cv_df)
print_top_terms(n_components=n_components, topics=topics, terms=cv.get_feature_names_out())

Topic 0:	['plug', 'ettin', 'ac', 'jet', 'lighting', 'crumple', 'rate', 'encase', 'disarm', 'closet']
Topic 1:	['resource', 'underling', 'singular', 'advance', 'constantly', 'clasp', 'tax', 'male', 'temperature', 'advisor']
Topic 2:	['bush', 'leper', 'mistress', 'pebble', 'wire', 'shank', 'stat', 'stabilize', 'nudge', 'occur']
Topic 3:	['easy', 'scrying', 'phrase', 'scrap', 'octopus', 'levitate', 'bracer', 'topple', 'country', 'orchard']
Topic 4:	['awareness', 'organize', 'inn', 'bump', 'poo', 'bastion', 'nightmare', 'kneel', 'advice', 'stain']
Topic 5:	['whale', 'torrent', 'hiding', 'regain', 'clatter', 'crouch', 'headmaster', 'goes', 'speck', 'thousand']
Topic 6:	['harel', 'ank', 'aura', 'molasse', 'bladestorm', 'squeak', 'holding', 'y', 'component', 'shackle']
Topic 7:	['kima', 'assign', 'higher', 'activate', 'gray', 'situate', 'import', 'rag', 'crop', 'bath']
Topic 8:	['parkour', 'brevity', 'running', 'whooshing', 'worshiper', 'wig', 'sudden', 'shaft', 'spine', 'warhammer']
Topic 9:

In [34]:
lda = LatentDirichletAllocation(n_components=n_components)
topics = lda.fit_transform(tfidf_df)
print_top_terms(n_components=n_components, topics=topics, terms=tfidf.get_feature_names_out())

Topic 0:	['onyx', 'snack', 'stealthily', 'role', 'dead', 'scrying', 'judgment', 'bargain', 'lure', 'warn']
Topic 1:	['reflection', 'r', 'trash', 'whale', 'jug', 'clicking', 'obscure', 'clatter', 'lair', 'merch']
Topic 2:	['correct', 'cap', 'relic', 'spinning', 'inform', 'tether', 'boost', 'score', 'closet', 'warhammer']
Topic 3:	['invest', 'hashtag', 'organization', 'wax', 'candy', 'submerge', 'black', 'dmed', 'snitch', 'percent']
Topic 4:	['blacken', 'cradle', 'deny', 'encampment', 'drinking', 'wire', 'chatroom', 'beetle', 'deadeye', 'eyeball']
Topic 5:	['rotate', 'tighten', 'stress', 'core', 'backhand', 'stomp', 'pepper', 'alignment', 'yank', 'drown']
Topic 6:	['gag', 'collar', 'lung', 'compare', 'furniture', 'kima', 'kneel', 'corrupt', 'spoil', 'alert']
Topic 7:	['snore', 'commission', 'permission', 'erase', 'weaken', 'fortress', 'ew', 'exhaustion', 'poo', 'text']
Topic 8:	['drawing', 'killer', 'midday', 'triceratop', 'hissing', 'discern', 'horribly', 'necromancy', 'flavor', 'disarm

## SVD

In [35]:
svd = TruncatedSVD(n_components=n_components)
topics = svd.fit_transform(cv_df)
print_top_terms(n_components=n_components, topics=topics, terms=cv.get_feature_names_out())

Topic 0:	['contemplate', 'ware', 'cling', 'sage', 'breeze', 'port', 'creator', 'drape', 'ceremony', 'looter']
Topic 1:	['practitioner', 'greed', 'hoard', 'narrowly', 'mercy', 'contemplate', 'cling', 'creator', 'ceremony', 'looter']
Topic 2:	['conclude', 'installation', 'inhale', 'invoke', 'parameter', 'eternity', 'completion', 'obligation', 'scribe', 'recipient']
Topic 3:	['flying', 'hike', 'manipulation', 'tradition', 'knack', 'meditate', 'childhood', 'truly', 'impression', 'headmaster']
Topic 4:	['divergence', 'den', 'potential', 'abdoman', 'devise', 'orcs', 'puzzle', 'reign', 'demand', 'devour']
Topic 5:	['intensity', 'brighten', 'apex', 'chip', 'atop', 'stain', 'account', 'stillness', 'entryway', 'sanctuary']
Topic 6:	['recede', 'intersection', 'amphitheater', 'desperation', 'crossroad', 'speech', 'prayer', 'bond', 'erathis', 'bench']
Topic 7:	['snuck', 'cobra', 'midst', 'boon', 'purely', 'follower', 'carriage', 'masquerade', 'ti', 'yuan']
Topic 8:	['unfold', 'user', 'origin', 'sub

In [36]:
svd = TruncatedSVD(n_components=n_components)
topics = svd.fit_transform(tfidf_df)
print_top_terms(n_components=n_components, topics=topics, terms=tfidf.get_feature_names_out())

Topic 0:	['intentionally', 'gingerly', 'sizzle', 'yesterday', 'hiding', 'instantly', 'score', 'mimic', 'socket', 'eyeball']
Topic 1:	['disease', 'sure', 'inhale', 'missile', 'drown', 'yesterday', 'sailor', 'warhammer', 'locket', 'grieve']
Topic 2:	['bless', 'mini', 'toast', 'tick', 'queen', 'invisibility', 'lycan', 'gingerly', 'locket', 'theater']
Topic 3:	['accord', 'tearfully', 'advisor', 'goes', 'deadeye', 'vomit', 'marching', 'toast', 'gingerly', 'locket']
Topic 4:	['levitate', 'tary', 'disease', 'yesterday', 'tick', 'sizzle', 'sharpshooter', 'locket', 'invisibility', 'lycan']
Topic 5:	['dmed', 'queen', 'disarm', 'sure', 'y', 'keg', 'tary', 'vomit', 'gingerly', 'lycan']
Topic 6:	['sizzle', 'warn', 'gone', 'disarm', 'grieve', 'vomit', 'lately', 'tary', 'gingerly', 'invisibility']
Topic 7:	['casino', 'toast', 'lycan', 'locket', 'text', 'yesterday', 'lately', 'mimic', 'vomit', 'pen']
Topic 8:	['theater', 'disperse', 'stabilize', 'locket', 'gone', 'vomit', 'healer', 'lately', 'yesterda

## NMF

In [37]:
nmf = NMF(n_components=n_components, max_iter=500)
topics = nmf.fit_transform(cv_df)
print_top_terms(n_components=n_components, topics=topics, terms=cv.get_feature_names_out())

/home/patrick/anaconda3/envs/metis/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


Topic 0:	['barker', 'storefront', 'combine', 'nostril', 'drape', 'necessity', 'breeze', 'ware', 'sage', 'port']
Topic 1:	['creator', 'practitioner', 'greed', 'hoard', 'narrowly', 'mercy', 'contemplate', 'ceremony', 'cling', 'looter']
Topic 2:	['conclude', 'installation', 'inhale', 'invoke', 'parameter', 'eternity', 'completion', 'obligation', 'scribe', 'recipient']
Topic 3:	['childhood', 'expectation', 'tradition', 'flying', 'hike', 'manipulation', 'meditate', 'truly', 'impression', 'headmaster']
Topic 4:	['nameless', 'divergence', 'potential', 'den', 'abdoman', 'orcs', 'puzzle', 'reign', 'demand', 'devour']
Topic 5:	['harry', 'stain', 'chip', 'recovery', 'brighten', 'guardian', 'kneel', 'stillness', 'entryway', 'sanctuary']
Topic 6:	['recede', 'intersection', 'amphitheater', 'desperation', 'crossroad', 'speech', 'prayer', 'bond', 'erathis', 'bench']
Topic 7:	['creator', 'purely', 'commotion', 'snuck', 'cobra', 'midst', 'boon', 'follower', 'ti', 'yuan']
Topic 8:	['unfold', 'subscribe',